# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [83]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
 
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_fscore_support,accuracy_score,label_ranking_average_precision_score
from sklearn.model_selection  import GridSearchCV
import re

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /home/jey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [84]:
# # load data from database
# engine = create_engine('sqlite:///InsertDatabaseName.db')
# df = 
# X = 
# Y = # load data from database
engine = create_engine('sqlite:///DisasterResponseJupter.db')

inspector = inspect(engine)

# Get table information
print(inspector.get_table_names())
df = pd.read_sql("SELECT * FROM Message", engine)
# X = 
# Y = 
df.head()

['Message']


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
X_names = ['message']
X = df['message'].values
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [86]:
Y_names = list(set(df.columns.values) - set( ['id','message', 'original', 'genre']))
Y = df[Y_names]
Y

,child_alone,floods,medical_help,direct_report,buildings,military,food,infrastructure_related,other_weather,electricity,...,medical_products,refugees,security,storm,search_and_rescue,aid_related,hospitals,water,weather_related,death
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
7,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,1,1,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0


In [87]:
category_names = Y.columns.values
category_names

array(['child_alone', 'floods', 'medical_help', 'direct_report',
       'buildings', 'military', 'food', 'infrastructure_related',
       'other_weather', 'electricity', 'aid_centers', 'fire', 'money',
       'tools', 'missing_people', 'offer', 'cold', 'related', 'shelter',
       'shops', 'earthquake', 'transport', 'request',
       'other_infrastructure', 'other_aid', 'clothing',
       'medical_products', 'refugees', 'security', 'storm',
       'search_and_rescue', 'aid_related', 'hospitals', 'water',
       'weather_related', 'death'], dtype=object)

In [88]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            62G         16G         45G        240K        1.3G         46G
Swap:          975M        319M        656M


### 2. Write a tokenization function to process your text data

In [89]:
def tokenize(text):
    #init stopwords and WordNetLemmatizer
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
     
    # tokenize text
    tokens = word_tokenize(text)
     
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
 
    return tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [100]:

pipeline =  Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidfvect', TfidfVectorizer(tokenizer=tokenize, stop_words='english')),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])


In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
X_train

array(['i am a victim of the earthquake. i need some help and money so i can go back home. I could lease a ',
       "we need help in food because our house is crush, and we don't have anyone to take care of us. ",
       "Diplomats have been more concerned about an upsurge of violence in the island's embattled east where at least 10 people were killed since Saturday in apparent internecine clashes.",
       ...,
       'JAKARTA, March 31, 2007 (AFP) - Indonesia\'s state oil and gas company Pertamina said Saturday it had shut off gas lines near a "mud volcano" for safety reasons, as railways and roads were again flooded with toxic sludge.',
       'Although the wet season was supposed to have ended in March, heavy rain fell near the South Jakarta-Tangerang border in Bintaro and Bumi Serpong Damai on Thursday, causing flooding as high as 40 centimeters on the toll road linking the two residential areas.',
       'JOHANNESBURG, SOUTH AFRICA - Helicopters raced to save Mozambican villager

In [102]:
y_train

,child_alone,floods,medical_help,direct_report,buildings,military,food,infrastructure_related,other_weather,electricity,...,medical_products,refugees,security,storm,search_and_rescue,aid_related,hospitals,water,weather_related,death
1127,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
7560,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
25227,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17552,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,1,0
20769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8063,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13705,0,1,0,0,1,0,0,0,0,0,...,0,1,0,1,0,1,0,0,1,0
18497,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1522,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [103]:
pipeline.fit(X_train, y_train)

/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jey/anaconda3/lib/python3.

/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('tfidfvect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=T...           oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [104]:
y_preds = pd.DataFrame(pipeline.predict(X_test), columns = category_names)
y_preds.head()

,child_alone,floods,medical_help,direct_report,buildings,military,food,infrastructure_related,other_weather,electricity,...,medical_products,refugees,security,storm,search_and_rescue,aid_related,hospitals,water,weather_related,death
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
y_test.shape, y_preds.shape

((6554, 36), (6554, 36))

In [106]:
def evaluate_mode(model, X_test, y_test, col_name):
    """ Evaluate model on test set,
        Predict results for each category.
        
    Args:
        model: trained model
        X_test: pandas.DataFrame for predict 
        y_test: pandas.DataFrame for labeled test set
        col_name: list for category names
        `
    Returns: 
        none.
    
    """
    
    # predict test df
    Y_pred = model.predict(X_test)
    tot_acc = 0
    tot_f1 = 0
    # print report 
    for i, cat in enumerate(col_name):    
        metrics =  classification_report(y_test[y_test.columns[i]], Y_pred[:,i])
        tot_acc += accuracy_score(y_test[y_test.columns[i]], Y_pred[:,i])
        tot_f1 += precision_recall_fscore_support(y_test[y_test.columns[i]], Y_pred[:,i], average = 'weighted')[2]
        print(cat, 'accuracy: {:.2f}'.format(accuracy_score(y_test[y_test.columns[i]], Y_pred[:,i])))
        print(metrics)
    print('total accuracy {:.2f}'.format(tot_acc/len(col_name)))
    print('total f1 {:.2f}'.format(tot_f1/len(col_name)))


In [107]:
evaluate_mode(pipeline, X_test, y_test, category_names)

child_alone accuracy: 1.00
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

   micro avg       1.00      1.00      1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

floods accuracy: 0.94
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6006
           1       0.88      0.36      0.51       548

   micro avg       0.94      0.94      0.94      6554
   macro avg       0.91      0.68      0.74      6554
weighted avg       0.94      0.94      0.93      6554

medical_help accuracy: 0.92
              precision    recall  f1-score   support

           0       0.92      0.99      0.96      5997
           1       0.53      0.10      0.17       557

   micro avg       0.92      0.92      0.92      6554
   macro avg       0.73      0.55      0.56      6554
weighted avg       0.89      0.92      0.89      65

/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


money accuracy: 0.98
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6394
           1       0.60      0.02      0.04       160

   micro avg       0.98      0.98      0.98      6554
   macro avg       0.79      0.51      0.51      6554
weighted avg       0.97      0.98      0.96      6554

tools accuracy: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6513
           1       0.00      0.00      0.00        41

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

missing_people accuracy: 0.99
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6479
           1       0.67      0.03      0.05        75

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.83      0.51      0.52      6554
we

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'tfidfvect__ngram_range': [(1, 1), (1, 2)],
        'tfidfvect__max_df': [0.75, 1.0],
        'tfidfvect__max_features': [2000, 5000],
        'clf__estimator__n_estimators': [10, 15],
        'clf__estimator__max_depth': [None, 15],
        'clf__estimator__min_samples_split': [2, 5]
        }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv_model = cv.fit(X_train, y_train)


/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated 

/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not in stop_words.
  sorted(inconsistent))
/home/jey/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['le', 'u'] not

In [ ]:
evaluate_mode(cv_model, X_test, y_test, Y.columns)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.